# Query PostgreSQL for user_id = 111

This notebook queries all tables in the PostgreSQL database for records with `user_id = 111` and exports the results to CSV and JSON files.

In [1]:
import os
from pathlib import Path

import pandas as pd
import psycopg2
from dotenv import load_dotenv
from psycopg2.extras import RealDictCursor

# Load environment variables from project root
project_root = Path.cwd().parent
env_path = project_root / ".env"
load_dotenv(env_path)

print(f"Loading environment from: {env_path}")
print(f"Environment loaded: {env_path.exists()}")

Loading environment from: /Users/danielkim/_Capstone/PrepSense/.env
Environment loaded: True


In [ ]:
# Database connection parameters
DB_HOST = os.getenv("POSTGRES_HOST")
DB_PORT = os.getenv("POSTGRES_PORT", 5432)
DB_NAME = os.getenv("POSTGRES_DATABASE")
DB_USER = os.getenv("POSTGRES_USER")
DB_PASSWORD = os.getenv("POSTGRES_PASSWORD")

print(f"DB_HOST: {DB_HOST}")
print(f"DB_PORT: {DB_PORT}")
print(f"DB_NAME: {DB_NAME}")
print(f"DB_USER: {DB_USER}")
print(f'DB_PASSWORD: {"*" * len(DB_PASSWORD) if DB_PASSWORD else None}')

# User ID to query for
USER_ID = 111

DB_HOST: 35.184.61.42
DB_PORT: 5432
DB_NAME: prepsense
DB_USER: postgres
DB_PASSWORD: None


In [18]:
# Connect to PostgreSQL
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)

print("Connected to PostgreSQL successfully!")
conn.autocommit = True

OperationalError: connection to server at "35.184.61.42", port 5432 failed: fe_sendauth: no password supplied


In [5]:
# Find all tables with a user_id column
with conn.cursor(cursor_factory=RealDictCursor) as cur:
    cur.execute("""
        SELECT table_schema, table_name
        FROM information_schema.columns
        WHERE column_name = 'user_id'
        AND table_schema NOT IN ('information_schema', 'pg_catalog')
        ORDER BY table_schema, table_name;
    """)
    tables = [(row["table_schema"], row["table_name"]) for row in cur.fetchall()]

if tables:
    print("Tables with user_id:")
    for schema, table in tables:
        print(f"- {schema}.{table}")
len(tables)

Tables with user_id:
- public.ai_recipe_cache
- public.food_categorization_corrections
- public.food_search_history
- public.pantries
- public.pantry_consumption_trends
- public.pantry_history
- public.pantry_transaction_logs
- public.recipe_completion_analytics
- public.recipe_completion_summary
- public.recipe_completions
- public.search_query_embeddings
- public.shopping_list_items
- public.spoonacular_api_calls
- public.user_allergens
- public.user_cuisine_preferences
- public.user_dietary_preferences
- public.user_pantry_full
- public.user_preferences
- public.user_recipes
- public.user_shopping_list_summary
- public.users


21

In [6]:
export_dir = Path("exports")
export_dir.mkdir(exist_ok=True)
export_dir.resolve()

PosixPath('/Users/danielkim/_Capstone/PrepSense/notebooks/exports')

In [12]:
def query_table(connection, schema: str, table: str, user_id: int) -> pd.DataFrame:
    sql = f'SELECT * FROM {schema}."{table}" WHERE user_id = %s'
    with connection.cursor(cursor_factory=RealDictCursor) as cur:
        cur.execute(sql, (user_id,))
        rows = cur.fetchall()
    return pd.DataFrame(rows)

summaries = []
for schema, table in tables:
    try:
        df = query_table(conn, schema, table, USER_ID)
        if df.empty:
            print(f"No rows for user {USER_ID} in {schema}.{table}")
            continue
        csv_path = export_dir / f"{schema}.{table}.user_{USER_ID}.csv"
        json_path = export_dir / f"{schema}.{table}.user_{USER_ID}.json"
        df.to_csv(csv_path, index=False)
        df.to_json(json_path, orient="records", indent=2, date_format="iso")
        print(f"Exported {len(df)} rows from {schema}.{table} -> {csv_path.name}, {json_path.name}")
        summaries.append({"schema": schema, "table": table, "rows": len(df)})
    except Exception as e:
        print(f"Error querying {schema}.{table}: {e}")

pd.DataFrame(summaries).sort_values(["schema","table"]).reset_index(drop=True) if summaries else pd.DataFrame(columns=["schema","table","rows"])

Error querying public.ai_recipe_cache: connection already closed
Error querying public.food_categorization_corrections: connection already closed
Error querying public.food_search_history: connection already closed
Error querying public.pantries: connection already closed
Error querying public.pantry_consumption_trends: connection already closed
Error querying public.pantry_history: connection already closed
Error querying public.pantry_transaction_logs: connection already closed
Error querying public.recipe_completion_analytics: connection already closed
Error querying public.recipe_completion_summary: connection already closed
Error querying public.recipe_completions: connection already closed
Error querying public.search_query_embeddings: connection already closed
Error querying public.shopping_list_items: connection already closed
Error querying public.spoonacular_api_calls: connection already closed
Error querying public.user_allergens: connection already closed
Error querying pub

,schema,table,rows


In [11]:
# Optional: peek into key known tables if present
key_tables = [
    ("public", "pantry_items"),
    ("public", "user_recipes"),
    ("public", "recipe_consumption"),
    ("public", "shopping_lists"),
    ("public", "user_preferences")
]

present = {(s,t) for (s,t) in tables}
for s, t in key_tables:
    if (s, t) in present:
        try:
            df = query_table(conn, s, t, USER_ID)
            print(f"\n{s}.{t} preview ({len(df)} rows):")
            print(df.head().to_string())
        except Exception as e:
            print(f"Error previewing {s}.{t}: {e}")

Error previewing public.user_recipes: connection already closed
Error previewing public.user_preferences: connection already closed


In [9]:
conn.close()
print("Database connection closed.")

Database connection closed.


## Done
- Exports saved in `notebooks/exports/` relative to this notebook.
- Adjust `USER_ID` or env variables as needed and re-run.